In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import datetime

%matplotlib inline


reading in the data:

In [2]:
members = pd.read_csv('../data/NSS_GreatCare_Members.csv')
ipadmit = pd.read_csv('../data/NSS_Hospital_Inpatient_Admissions.csv')
pcp = pd.read_csv('../data/NSS_PrimaryCare_Office_Visits.csv', low_memory = False)

In [3]:
members.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61519 entries, 0 to 61518
Data columns (total 63 columns):
 #   Column                                                           Non-Null Count  Dtype  
---  ------                                                           --------------  -----  
 0   Patient ID                                                       61519 non-null  object 
 1   Primary Plan ID                                                  61519 non-null  object 
 2   Primary Payor ID                                                 61519 non-null  object 
 3   Is Patient Alive                                                 61519 non-null  int64  
 4   Patient Age Integer                                              61519 non-null  int64  
 5   Patient Date of Birth                                            61519 non-null  object 
 6   Patient ZIP Code 5-Digit                                         61105 non-null  float64
 7   Count Primary Care Visits in Last 36 Mon

In [18]:
ipadmit.info()
ipadmit.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6213 entries, 0 to 6212
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Encounter ID           6213 non-null   object
 1   Patient ID             6213 non-null   object
 2   Facility Provider NPI  6213 non-null   object
 3   Encounter Start Date   6213 non-null   object
 4   Encounter End Date     6213 non-null   object
dtypes: object(5)
memory usage: 242.8+ KB


,Encounter ID,Patient ID,Facility Provider NPI,Encounter Start Date,Encounter End Date
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,1306890389,2013-01-12,2013-01-14
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,1902803315,2014-01-05,2014-01-12
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,1902803315,2012-06-13,2012-06-21
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,1427055839,2012-12-06,2012-12-09
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,1992818256,2015-03-11,2015-03-14
5,HIA|DS1|10126|DS1|NPI|1902803315|1,DS1|10126,1902803315,2010-04-07,2010-04-09
6,HIA|DS1|10130|DS1|NPI|1427055839|1,DS1|10130,1427055839,2014-06-01,2014-06-05
7,HIA|DS1|10132|DS1|NPI|1366545311|1,DS1|10132,1366545311,2013-08-14,2013-08-16
8,HIA|DS1|10132|DS1|NPI|1386741635|1,DS1|10132,1386741635,2013-03-30,2013-04-01
9,HIA|DS1|10135|DS1|NPI|1184722779|1,DS1|10135,1184722779,2013-08-20,2013-08-22


In [14]:
pcp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231373 entries, 0 to 231372
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Patient ID      231373 non-null  object
 1   Encounter Date  231373 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


In [26]:
ipadmit[['Encounter Start Date','Encounter End Date']] = ipadmit[['Encounter Start Date','Encounter End Date']].apply(pd.to_datetime)
ipadmit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6213 entries, 0 to 6212
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Encounter ID           6213 non-null   object        
 1   Patient ID             6213 non-null   object        
 2   Facility Provider NPI  6213 non-null   object        
 3   Encounter Start Date   6213 non-null   datetime64[ns]
 4   Encounter End Date     6213 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 242.8+ KB


In [27]:
pcp['Encounter Date'] = pd.to_datetime(pcp['Encounter Date'])
pcp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231373 entries, 0 to 231372
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Patient ID      231373 non-null  object        
 1   Encounter Date  231373 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 3.5+ MB


<ipython-input-27-b26902b01edc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pcp['Encounter Date'] = pd.to_datetime(pcp['Encounter Date'])


Question 1 What is our 30-day all cause readmission rate across hospitals our members are admitted to?

In [28]:
ipadmit = ipadmit[['Encounter ID','Patient ID','Facility Provider NPI','Encounter Start Date','Encounter End Date']]

In [23]:
#demoninatorr = 6213
ipadmit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6213 entries, 0 to 6212
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Encounter ID           6213 non-null   object        
 1   Patient ID             6213 non-null   object        
 2   Facility Provider NPI  6213 non-null   object        
 3   Encounter Start Date   6213 non-null   datetime64[ns]
 4   Encounter End Date     6213 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 242.8+ KB


In [36]:
#I wanted to be able to do a count of total admissions with duplicates...took me a while to figure it out...thanks Brenda!
#need it so I can use code for question 1
ipadmit['Patient ID'].count()

6213

In [35]:
ipadmit = ipadmit.sort_values(['Patient ID',"Encounter Start Date", "Encounter End Date"], ascending = (False, True, True))
ipadmit

,Encounter ID,Patient ID,Facility Provider NPI,Encounter Start Date,Encounter End Date
6212,HIA|DS1|9996|DS1|NPI|1033298617|1,DS1|9996,1033298617,2010-03-28,2010-03-31
6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,1689608150,2015-11-21,2015-11-23
6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,1689608150,2015-11-24,2015-11-27
6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,1427055839,2014-08-07,2014-08-10
6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,1689772592,2015-02-06,2015-02-11
...,...,...,...,...,...
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,1427055839,2012-12-06,2012-12-09
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,1902803315,2012-06-13,2012-06-21
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,1902803315,2014-01-05,2014-01-12
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,1306890389,2013-01-12,2013-01-14


In [37]:
one_top = ipadmit[['Patient ID', 'Previous End Date', 'Encounter Start Date', 'Encounter End Date', 'Days Since Last Visit']]
one_top = ipadmit[one_top['Days Since Last Visit'].notnull()]
one_top = ipadmit[one_top['Days Since Last Visit'] <= '30 days']
one_top = ipadmit[one_top['Days Since Last Visit'] >= '0 days']
one_top
read_count = len(one_top)
read_count
one_top

KeyError: "['Days Since Last Visit', 'Previous End Date'] not in index"

In [ ]:
t = []

for i in multistay['Days Between']:
    if (i <= dt.timedelta(days=30)) and (i >= dt.timedelta(days=0)) :
        t.append('yes')
    else:
        t.append('no')

multistay['Under 30 Days'] = t
multistay

In [11]:
# pcp = pcp[['Patient ID','Encounter Date']]

In [12]:
# pcp

,Patient ID,Encounter Date
0,DS1|10076,2011-02-18
1,DS1|10088,2014-02-22
2,DS1|10165,2014-08-31
3,DS1|10240,2013-06-17
4,DS1|10293,2016-09-04
...,...,...
231368,DS1|9999,2015-02-10
231369,DS1|99,2011-08-02
231370,DS1|99,2011-04-13
231371,DS1|9,2014-04-07


merge these two dataframes:


In [29]:
# admit_pcp= ipadmit.merge(pcp, how = 'left', on='Patient ID')
# admit_pcp

,Encounter ID,Patient ID,Facility Provider NPI,Encounter Start Date,Encounter End Date,Encounter Date
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,1306890389,2013-01-12,2013-01-14,2015-12-08
1,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,1306890389,2013-01-12,2013-01-14,2012-06-18
2,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,1306890389,2013-01-12,2013-01-14,2014-06-10
3,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,1306890389,2013-01-12,2013-01-14,2014-09-28
4,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,1306890389,2013-01-12,2013-01-14,2014-03-16
...,...,...,...,...,...,...
44093,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,1689608150,2015-11-24,2015-11-27,2015-12-05
44094,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,1689608150,2015-11-24,2015-11-27,2015-12-13
44095,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,1689608150,2015-11-24,2015-11-27,2014-05-18
44096,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,1689608150,2015-11-24,2015-11-27,2013-07-20
